### WebCrawing
- 웹페이지의 종류
    - 정적페이지: 페이지의 데이터가 변경될때 URL이 변경 O -> html 수집
    https://news.naver.com/
    - 동적페이지: 페이지의 데이터가 변경될때 URL이 변경 X -> json 수집
    페이지내에 더보기 같은 경우, json으로 데이터를 받아와 상황에 따라 데이터 출력

- requests package
    - 브라우저의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력 : URL -> DATA
    - requests 패키지 : URL -> DATA

### Naver Stock Data
- Kospi 지수
- Kosdaq 지수
- USD : 원달러 환율

In [4]:
import requests
import pandas as pd

### 1. 웹서비스를 분석 : 크롬 개발자 도구 : URL

https://m.stock.naver.com/domestic/index/KOSPI

In [86]:
url = "https://m.stock.naver.com/api/index/KOSPI/price?pageSize=20&page=6"

### 2. request > response : JSON(str)

In [87]:
response = requests.get(url)
response

<Response [200]>

In [106]:
response.text[0:200]

'[{"localTradedAt":"2022-07-06","closePrice":"744.63","compareToPreviousClosePrice":"-6.32","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-0.84","openPrice":"7'

### 3. JSON(str) > list, dict > DataFrame

In [108]:
data = response.json()
type(response), data[0]

(requests.models.Response,
 {'localTradedAt': '2022-07-06',
  'closePrice': '744.63',
  'compareToPreviousClosePrice': '-6.32',
  'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
  'fluctuationsRatio': '-0.84',
  'openPrice': '750.04',
  'highPrice': '759.88',
  'lowPrice': '743.61'})

In [92]:
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
df.tail(2)

,localTradedAt,closePrice
18,2022-02-11,"2,747.71"
19,2022-02-10,"2,771.93"


### 4. 함수만들기
### param : pagesize, page

In [27]:
def stock_price(pagesize, page):
    url = f"https://m.stock.naver.com/api/index/KOSPI/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]

In [35]:
df = stock_price(100, 5)
df.tail(2)

,localTradedAt,closePrice
8,2022-05-11,"2,592.27"
9,2022-05-10,"2,596.56"


    # KOSDQ 데이터 수집 코드 작성

### 1 웹서비스 분석 : URL

In [100]:
url = "https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=3"

### 2. request > response : JSON(str)

In [101]:
response = requests.get(url)
data = response.json()

### 3. JSON(str) > list, dict > DataFrame

In [102]:
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
df

,localTradedAt,closePrice
0,2022-07-06,744.63
1,2022-07-05,750.95
2,2022-07-04,722.73
3,2022-07-01,729.48
4,2022-06-30,745.44
5,2022-06-29,762.35
6,2022-06-28,769.51
7,2022-06-27,770.60
8,2022-06-24,750.30
9,2022-06-23,714.38



### 4. 함수만들기
### param : pagesize, page

In [119]:
def stock_price(pagesize, page, code):
    """
    This function is crwaling stock price from naver stock web page
    
    parameters :
        page_size : int : one page size
        page : int : page number
        code : str : KOSPI or KOSDAQ
    
    return :
        type : DataFame of pandas
    """
    
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()

    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]

In [105]:
df = stock_price(20, 1, "KOSDAQ")
df.tail(2)

,localTradedAt,closePrice
18,2022-07-08,766.48
19,2022-07-07,757.97


In [ ]:
### stock_price(10, 1, "KOSDAQ")

In [109]:
kospi = stock_price(60, 1, "KOSPI")
kosdaq = stock_price(60, 1, "KOSDAQ")

### docstring

In [120]:
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code)
    This function is crwaling stock price from naver stock web page
    
    parameters :
        page_size : int : one page size
        page : int : page number
        code : str : KOSPI or KOSDAQ
    
    return :
        type : DataFame of pandas



### 원달러 환율 데이터 수집
https://m.stock.naver.com/ > 시장지표 > 미국 USD > 일별시세(더보기)

In [121]:
### 최근 60일 원달러 환율 데이터 수집

In [125]:
### 1. 웹서비스 분석 : URL

In [139]:
page = 1
pagesize = 60
url = f"https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page={page}&pageSize={pagesize}"
response = requests.get(url)
response

<Response [200]>

In [140]:
### 2. request(url) > response(json) : JSON(str)

In [141]:
data = response.json()
data[0]

{'localTradedAt': '2022-08-03',
 'closePrice': '1,309.70',
 'fluctuations': '-2.30',
 'fluctuationsRatio': '-0.18',
 'fluctuationsType': {'code': '5', 'text': '하락', 'name': 'FALLING'},
 'cashBuyValue': '1,332.61',
 'cashSellValue': '1,286.79',
 'sendValue': '1,322.50',
 'receiveValue': '1,296.90'}

In [142]:
### 3. JSON(str) > list, dict > DataFrame

In [146]:
df = pd.DataFrame(data)
df.tail(2)

,localTradedAt,closePrice,fluctuations,fluctuationsRatio,fluctuationsType,cashBuyValue,cashSellValue,sendValue,receiveValue
58,2022-05-11,"1,274.50",-1.50,-0.12,"{'code': '5', 'text': '하락', 'name': 'FALLING'}","1,296.80","1,252.20","1,286.90","1,262.10"
59,2022-05-10,"1,276.00",-1.50,-0.12,"{'code': '5', 'text': '하락', 'name': 'FALLING'}","1,298.33","1,253.67","1,288.50","1,263.50"


In [147]:
def stock_price(pagesize, page, code):
    """
    This function is crwaling stock price from naver stock web page
    
    parameters :
        page_size : int : one page size
        page : int : page number
        code : str : FX_USDKRW or FX_EURKRW
    
    return :
        type : DataFame of pandas
    """
    
    url = f"https://api.stock.naver.com/marketindex/exchange/{code}/prices?page={page}&pageSize={pagesize}"
    response = requests.get(url)
    data = response.json()

    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]

In [153]:
usd = stock_price(60, 1, "FX_USDKRW")
usd.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


### 데이터 분석
#### 상관관계분석 : 두 데이터 집합 사이에 어떤 관계가 있는지 확인하는 분석방법
#### 원달러 환율이 높으면 코스피, 코스닥 지수가 낮다. > 음의 상관관계를 갖는다.

### 피어슨 상관계수 : df.corr()
#### 1과 가까울수록 강한 양의 상관관계를 갖는다.
#### -1과 가까울수록 강한 음의 상관관계를 갖는다.
#### 0과 가까울수록 관계가 없다.

In [184]:
### 데이터 전처리
df = kospi.copy()
df["kosdaq"] = kosdaq["closePrice"]
df["usd"] = usd["closePrice"]
df = df.rename(columns={"closePrice" : "kospi"})
df = df.drop(columns=["localTradedAt"])
df.tail(2)

,kospi,kosdaq,usd
58,"2,592.27",866.34,"1,274.50"
59,"2,596.56",856.14,"1,276.00"


In [195]:
### 칼럼의 데이터 타입 변경 str -> float
### df[column].apply() : 모든 데이터를 함수에 대입한 결과를 출력
df["kospi"] = df["kospi"].apply(lambda data : float(data.replace(",", "")))
df["kosdaq"] = df["kosdaq"].apply(lambda data : float(data.replace(",", "")))
df["usd"] = df["usd"].apply(lambda data : float(data.replace(",", "")))

In [196]:
df.dtypes

kospi     float64
kosdaq    float64
usd       float64
dtype: object

In [205]:
df[["kospi", "kosdaq", "usd"]].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984058,-0.878358
kosdaq,0.984058,1.000000,-0.821411
usd,-0.878358,-0.821411,1.000000


In [ ]:
# kospi <-> kosdaq 양의 상관 관계
# usd <-> kospi, kosdaq음의 상관 관계

### copy()

In [227]:
data1 = [1, 2, 3]

data2 = data1
data3 = data1.copy()
print(data1, data2, data3) # [1, 2, 3] [1, 2, 3]

data1[1] = 4
print(data1, data2, data3)

[1, 2, 3] [1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3] [1, 2, 3]


### apply()

In [228]:
df = pd.DataFrame([
    { "age" : 23 },
    { "age" : 36 },
    { "age" : 27 }
])
df

,age
0,23
1,36
2,27


In [229]:
# 연령대 컬럼을 추가 
def change_ages(age): 
    return age // 10 * 10

In [230]:
df["ages"] = df["age"].apply(change_ages)
display(df)

,age,ages
0,23,20
1,36,30
2,27,20


### lambda 일회성 함수
#### 사용이유 : 간단한 함수(파라미터를 받아서 바로 리턴하는)를 메모리를 절약하여 사용

In [231]:
def plus(n1, n2): 
    return n1 + n2

def minus(n1, n2): 
    return n1 - n2

def calc(func, n1, n2): 
    return func(n1, n2)

In [233]:
(calc(plus, 1, 2), calc(minus, 1, 2))

(3, -1)

In [235]:
plus_lambda = lambda n1, n2 : n1 + n2
(plus(2, 3), plus_lambda(2, 3))

(5, 5)

In [236]:
def calc(func, n1, n2): 
    return func(n1, n2)

(calc(lambda n1, n2 : n1 + n2, 1, 2), calc(lambda n1, n2 : n1 - n2, 1, 2))

(3, -1)

In [ ]:
# summary
# 웹페이지의 종류
# 정적페이지 : 데이터가 변경할때 URL 변경 O : HTML
# 동적페이지 : 데이터가 변경할때 URL 변경 X : JSON

# 웹크롤링 절차
# 1. 웹서비스 분석 : 크롬 개발자 도구 : URL
# 2. resquests(url) > response(json) : JSON(str)
# 3. JSON(str) > list, dict > DataFrame